# Solving a constrained Integer Linear Program using learned cost

We consider the problem of minimizing a constrained ILP program. The problem can be written formally as
\begin{equation}
\begin{aligned}
\mathbf{x}^{\ast} =& \mathop{\arg\min}_{\mathbf{x} \in \mathcal{X}} \mathbf{c(f;\mathbf{w})}^T \mathbf{x} \\
&\text{s.t.} \begin{aligned}[t]
     \mathbf{A}\mathbf{x} & = \mathbf{b} \\
     \mathbf{G}\mathbf{x} & \leq \mathbf{h}
  \end{aligned}
\end{aligned}
\end{equation}
where $\mathbf{c(f;\mathbf{w})} \in \mathbb{R}^n$ is the cost function parametrized by $\mathbf{\mathbf{w}}$, given input $\mathbf{f}$. And $\mathbf{A,b}$ and $\mathbf{G,h}$ defines the equality and in-equality constraints, respectively.

In [1]:
import numpy as np
np.set_printoptions(suppress=True)
import os, time
import sklearn
import pickle
import random
import cv2
import joblib
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)
from torch_geometric.data import Data
from lib.tracking import Tracker
from lib.utils import getIoU,computeBoxFeatures, trackletNMS, pruneTracks, interpolateTrack, interpolateTracks

class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Sequential(nn.Linear(6,6), nn.ReLU(), nn.Linear(6,1))
    def forward(self, data):
        x = self.fc(data.edge_attr)
        x = nn.Sigmoid()(x)
        return x
    
net = Net()
net.load_state_dict(torch.load('ckpt/qp/epoch_8.pth'))
#net.load_state_dict(torch.load('ckpt/qp/epoch_11.pth'))
tracker = Tracker(net)

def getTransitionProbability(tracker, curr_dets, curr_app_feats_norm, max_frame_gap = 1):
    """
    Get the transition cost of network flow, entry/exit and detection costs should be included later as a whole.
    tracker: an instance of the Tracker.
    curr_dets: np.ndarray, frame, x1, y1, x2, y2, conf, node_ind 
    curr_app_feats_norm: Nxd array, normalized appearance features for curr_det, N the number of detections.
    max_frame_gap: frame gap used to connect detections.
    """
    edge_ind, edge_feat_list = 0, []
    linkIndexGraph = np.zeros((curr_dets.shape[0], curr_dets.shape[0]), dtype=np.int32)
    for i in range(curr_dets.shape[0]):
        for j in range(curr_dets.shape[0]):
            frame_gap = curr_dets[j][0] - curr_dets[i][0]
            if frame_gap == max_frame_gap:
                box_feats = computeBoxFeatures(curr_dets[i, 1:5], curr_dets[j, 1:5])
                iou = getIoU(curr_dets[i, 1:5], curr_dets[j, 1:5])
                cos_sim = np.dot(curr_app_feats_norm[i], curr_app_feats_norm[j])
                box_feats.extend((iou, cos_sim))
                edge_feat_list.append(box_feats)
                edge_ind += 1
                linkIndexGraph[i, j] = edge_ind
    edge_feats = np.array(edge_feat_list)
    prob = tracker.net.fc(torch.from_numpy(edge_feats).float())
    prob = torch.clamp(prob, min=1e-07, max=1-1e-07)
    prob = prob.detach().numpy()
    return linkIndexGraph, prob

def getSkipTransitionProbability(curr_dets, curr_app_feats_norm, max_frame_gap = 5):
    """
    Handles false negatives(missing detections)
    Get the transition cost of network flow, entry/exit and detection costs should be included later as a whole.
    tracker: an instance of the Tracker.
    curr_dets: np.ndarray, frame, x1, y1, x2, y2, conf, node_ind 
    curr_app_feats_norm: nxd array, normalized appearance features for curr_dets, n is # of detections.
    max_frame_gap: frame gap used to connect detections.
    """
    edge_ind, edge_feat_list = 0, []
    linkIndexGraph = np.zeros((curr_dets.shape[0], curr_dets.shape[0]), dtype=np.int32)
    cos_sim_mat = np.dot(curr_app_feats_norm, curr_app_feats_norm.T)
    for i in range(curr_dets.shape[0]):
        for j in range(curr_dets.shape[0]):
            frame_gap = curr_dets[j][0] - curr_dets[i][0]
            cos_sim = cos_sim_mat[i, j]
            if frame_gap == 1:
                box_feats = computeBoxFeatures(curr_dets[i, 1:5], curr_dets[j, 1:5])
                iou = getIoU(curr_dets[i, 1:5], curr_dets[j, 1:5])
                box_feats.extend((iou, cos_sim, 1))
                edge_feat_list.append(box_feats)
                edge_ind += 1
                linkIndexGraph[i, j] = edge_ind
            elif frame_gap > 1 and frame_gap <= max_frame_gap:
                box_feats = 6 * [0]
                if cos_sim > 0.75:
                    time_weight = 0.9 ** frame_gap
                    box_feats.append(time_weight * cos_sim)
                else:
                    box_feats.append(-1)
                edge_feat_list.append(box_feats)
                edge_ind += 1
                linkIndexGraph[i, j] = edge_ind
                
    edge_feats = np.array(edge_feat_list)
    real_inds = np.where(edge_feats[:, -1] == 1)[0]
    skip_inds = np.where(np.logical_and(edge_feats[:, -1] != 1, edge_feats[:, -1] != -1))[0]
    prune_inds = np.where(edge_feats[:, -1] == -1)[0]
    
    prob_ = np.zeros(edge_feats.shape[0])
    _, prob = getTransitionProbability(tracker, curr_dets, curr_app_feats_norm)
    prob_[real_inds] = prob.squeeze()
    prob_[skip_inds] = edge_feats[:, -1][skip_inds]
    prob_[prune_inds] = 0
    return linkIndexGraph, prob_

# Tracking!

In [2]:
app_thresh = 0.75 #0.7, 0.8
nms_thresh, eps = 0.3, 1e-7

#for sequence in ['MOT17-01', 'MOT17-03', 'MOT17-06', 'MOT17-07', 'MOT17-08', 'MOT17-12', 'MOT17-14']:
for sequence in ['MOT17-01']:
    
    #Static camera, moving camera
    if sequence in ['MOT17-03']:
        batch_size, dist_thresh, prune_len = 50, 50, 2 #tracklets less than 2 are pruned
    else:
        batch_size, dist_thresh, prune_len = 100, 100, 3
        
    if sequence == 'MOT17-06':
        img_Height, img_Width = 480, 640
    else:
        img_Height, img_Width = 1080, 1920
        
    #for detector in ['DPM','FRCNN','SDP']:
    for detector in ['DPM']:
        print('Sequence {}, {} detection, app thresh {}, dist thresh {}, retain length {}'.format(
            sequence, detector, app_thresh, dist_thresh, prune_len))
        
        det_file = './data/_{}/det_{}.txt'.format(sequence, detector)
        app_file = './data/_{}/app_det_{}.npy'.format(sequence, detector)
        dets = np.loadtxt(det_file, delimiter=',')
        app_feats = np.load(app_file)
        assert dets.shape[0] == app_feats.shape[0], 'Shape mismatch'

        batch_overlap = 5                  # number of frames to overlap between 2 batches
        num_frames = int(dets[:, 0].max()) # number of frames in this sequence
        tracks_list, assignments_list, features_list, nms_list = [],[],[],[]
        
        for start_frame in range(1, num_frames+1, batch_size-batch_overlap):
            end_frame = start_frame + batch_size - 1
            if end_frame >= num_frames:
                end_frame = num_frames
            print('Tracking from frame %d to %d'%(start_frame, end_frame))
            curr_ind = np.logical_and(dets[:, 0] >= start_frame, dets[:, 0] <= end_frame)
            curr_dets = np.concatenate([dets[curr_ind, 0][:, None], dets[curr_ind, 2:7],
                                        np.arange(dets[curr_ind].shape[0])[:, None]], axis=1)

            curr_dets[:, 3:5] = curr_dets[:, 3:5] + curr_dets[:, 1:3] # convert to frame,x1,y1,x2,y2,conf,node_ind
            curr_app_feats = app_feats[curr_ind]
            curr_app_feats_norm = curr_app_feats / np.linalg.norm(curr_app_feats, axis=1, keepdims=True)
            for iteration in range(2):
                if iteration == 0:
                    print('%d-th iteration'%iteration)
                    linkIndexGraph, prob_ = getSkipTransitionProbability(curr_dets, curr_app_feats_norm, max_frame_gap=5)
                    transition_cost = - np.log(prob_+eps) #np.log((1-prob_+eps)/(prob+eps))
                    det_cost = -curr_dets[:, -2]
                    entry_cost = 0.5 * np.ones(det_cost.shape[0])
                    exit_cost = entry_cost
                    cost = np.concatenate((det_cost, entry_cost, exit_cost, transition_cost))

                    A_eq, b_eq, A_ub, b_ub = tracker.buildConstraint(linkIndexGraph)
                    sol = tracker.linprog(c=cost, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub)

                    tracklets = tracker.recoverTracklets(curr_dets, sol, linkIndexGraph, prune_len=prune_len)    
                    tracklets_ = np.delete(tracklets, -1, axis=1)
                    interpolated_tracklets = interpolateTracks(tracklets_)
                else:
                    print('%d-th iteration'%iteration)
                    assignment_list, feature_list = tracker.clusterSkipTracklets(tracklets, curr_app_feats_norm, dist_thresh, app_thresh)
                    tracks = tracker.recoverClusteredTracklets(tracklets, assignment_list)
                    tracks = interpolateTracks(tracks)

                    assignments_list.append(assignment_list)
                    feature_array = np.stack(feature_list)
                    feature_array = feature_array / np.linalg.norm(feature_array, axis=1, keepdims=True)

            tracks_list.append(tracks)
            features_list.append(feature_array)
            
        final_tracks = tracker.mergeTracklets(tracks_list, features_list)
        save_file = 'output/MOT17-{}-{}.txt'.format(sequence.split('-')[1], detector)
        print('Finished tracking, saving to {}'.format(save_file))
        np.savetxt(save_file, final_tracks, fmt='%d',delimiter=',')

Sequence MOT17-01, DPM detection, app thresh 0.75, dist thresh 100, retain length 3
Tracking from frame 1 to 100
0-th iteration
Using license file /home/lishuai/Experiment/gurobi910/gurobi.lic
Academic license - for non-commercial use only - expires 2021-03-09
1-th iteration
Tracking from frame 96 to 195
0-th iteration
1-th iteration
Tracking from frame 191 to 290
0-th iteration
1-th iteration
Tracking from frame 286 to 385
0-th iteration
1-th iteration
Tracking from frame 381 to 450
0-th iteration
1-th iteration


# Show final tracking and detection results

In [22]:
#sequence = 'MOT17-03'
detector = 'DPM'

for sequence in ['MOT17-01', 'MOT17-03', 'MOT17-06', 'MOT17-07', 'MOT17-08', 'MOT17-12', 'MOT17-14']:
    
    save_dir = 'BYTE_Results/{}-{}'.format(sequence, detector)
    print('save dir {}'.format(save_dir))
    os.makedirs(save_dir, exist_ok=True)
    
    #tracks: frame, ID, x, y, w, h, -1, -1, -1, -1
    #dets:   frame, -1, x, y, w, h, conf, -1, -1, -1
    tracks = np.loadtxt('BYTE_Results/MOT17-01-DPM.txt', delimiter=',')
    tracks = tracks.astype(np.int)
    
    colors = np.random.rand(1000,3)
    resize_scale = 0.5
    for frame in range(tracks[:, 0].min(), tracks[:, 0].max()+1):
        if frame % 100 == 0:
            print('Processing frame {}'.format(frame))
        
        img_file = os.path.join('/home/lishuai/Experiment/MOT/MOT17/test/{}-{}/img1/{:06d}.jpg'.format(sequence,detector,frame))
        img = cv2.imread(img_file)
        img = cv2.resize(img, (int(resize_scale*img.shape[1]), int(resize_scale*img.shape[0])))
        cv2.putText(img, '{:04}'.format(frame), (0,50) ,cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,0,255), thickness=2)
        bboxes = tracks[tracks[:, 0] == frame, 1:6]
        
        if bboxes.shape[0] != 0:
            #detections = dets[dets[:, 0] == frame, 2:7]
            for i in range(bboxes.shape[0]):
                ID = int(bboxes[i][0])
                x, y = int(resize_scale*(bboxes[i][1])), int(resize_scale*(bboxes[i][2]))
                w, h = int(resize_scale*(bboxes[i][3])), int(resize_scale*(bboxes[i][4]))
                cv2.rectangle(img, (x,y),(x+w,y+h), 255*colors[ID], thickness=2)
                cv2.putText(img, str(ID), (x,y) ,cv2.FONT_HERSHEY_SIMPLEX, 0.8, 255*colors[ID], thickness=2)

    #         for i in range(detections.shape[0]):
    #             x, y = int(resize_scale*(detections[i][0])), int(resize_scale*(detections[i][1]))
    #             w, h = int(resize_scale*(detections[i][2])), int(resize_scale*(detections[i][3]))
    #             score = detections[i][4] 
    #             drawrect(img,(x,y),(x+w,y+h),(0,0,255),2,'dotted')
        cv2.imwrite(save_dir+'/'+'{:06d}.jpg'.format(frame), img)

save dir BYTE_Results/MOT17-01-DPM
Processing frame 100
Processing frame 200
Processing frame 300
Processing frame 400
save dir BYTE_Results/MOT17-03-DPM
Processing frame 100
Processing frame 200
Processing frame 300
Processing frame 400
save dir BYTE_Results/MOT17-06-DPM
Processing frame 100
Processing frame 200
Processing frame 300
Processing frame 400
save dir BYTE_Results/MOT17-07-DPM
Processing frame 100
Processing frame 200
Processing frame 300
Processing frame 400
save dir BYTE_Results/MOT17-08-DPM
Processing frame 100
Processing frame 200
Processing frame 300
Processing frame 400
save dir BYTE_Results/MOT17-12-DPM
Processing frame 100
Processing frame 200
Processing frame 300
Processing frame 400
save dir BYTE_Results/MOT17-14-DPM
Processing frame 100
Processing frame 200
Processing frame 300
Processing frame 400
